# Median Data Preprocessing

## Purpose
Clean and merge rental property spreadsheets, then enrich suburb data with SA2/LGA regions.

## Inputs
- `Moving annual median rent by suburb and town - March quarter 2025.xlsx`  
- `mapped_target_suburbs.csv`  

## Outputs
- `all_properties_tidy.csv` – tidy suburb-month metrics  
- `all_properties_tidy_enriched.csv` – enriched with SA2/LGA info  
- `all_properties_tidy_UNMATCHED_SUBURBS.csv` – unmatched suburbs  

## Steps
1. Define project paths with `pathlib`.  
2. Split Excel sheets → individual CSVs.  
3. Strip headers and clean data.  
4. Rebuild headers, reshape to tidy format.  
5. Combine and export tidy CSV.  
6. Join with suburb mapping for SA2/LGA enrichment.  
7. Save enriched and unmatched outputs.  


In [ ]:
from pathlib import Path
import pandas as pd
import re
import os
import pandas as pd
import numpy as np
import re
from dateutil import parser as dateparser

In [22]:

# Project/data root
DATA_ROOT = Path("../../datasets")
RAW_DIR = DATA_ROOT / "raw"
SPLIT_DIR = RAW_DIR / "split_sheets"
STRIPPED_DIR = SPLIT_DIR / "split_sheets_stripped"
TIDY_DIR = RAW_DIR / "tidy_monthly"

# Excel and mapping files
EXCEL_PATH = DATA_ROOT / "property" / "Moving annual median rent by suburb and town - March quarter 2025.xlsx"
MAP_CSV = DATA_ROOT / "6. excel to sa2" / "mapped_target_suburbs.csv"

# Output files
OUT_TIDY = TIDY_DIR / "all_properties_tidy.csv"
OUT_ENRICHED = TIDY_DIR / "all_properties_tidy_enriched.csv"
OUT_UNMATCHED = TIDY_DIR / "all_properties_tidy_UNMATCHED_SUBURBS.csv"

# Create directories
for p in [SPLIT_DIR, STRIPPED_DIR, TIDY_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# Print paths
print("Paths:")
print("  EXCEL_PATH   =", EXCEL_PATH)
print("  MAP_CSV      =", MAP_CSV)
print("  SPLIT_DIR    =", SPLIT_DIR)
print("  STRIPPED_DIR =", STRIPPED_DIR)
print("  TIDY_DIR     =", TIDY_DIR)


Paths:
  EXCEL_PATH   = ../../datasets/property/Moving annual median rent by suburb and town - March quarter 2025.xlsx
  MAP_CSV      = ../../datasets/6. excel to sa2/mapped_target_suburbs.csv
  SPLIT_DIR    = ../../datasets/raw/split_sheets
  STRIPPED_DIR = ../../datasets/raw/split_sheets/split_sheets_stripped
  TIDY_DIR     = ../../datasets/raw/tidy_monthly


In [23]:
def _safe_name(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9]+", "_", s).strip("_").lower()

if not EXCEL_PATH.exists():
    raise FileNotFoundError(f"Excel not found: {EXCEL_PATH}\nCheck your PROJECT_ROOT or filename.")

 # needs openpyxl
xls = pd.ExcelFile(EXCEL_PATH) 
print("Found sheets:", xls.sheet_names)

for sheet in xls.sheet_names:
    df = pd.read_excel(EXCEL_PATH, sheet_name=sheet)
    out_csv = SPLIT_DIR / f"{_safe_name(sheet)}.csv"
    df.to_csv(out_csv, index=False)
    print(f"[split] Saved: {out_csv.name} ({len(df)} rows)")


Found sheets: ['1 bedroom flat', '2 bedroom flat', '3 bedroom flat', '2 bedroom house', '3 bedroom house', '4 bedroom house', 'All properties']
[split] Saved: 1_bedroom_flat.csv (161 rows)
[split] Saved: 2_bedroom_flat.csv (161 rows)
[split] Saved: 3_bedroom_flat.csv (161 rows)
[split] Saved: 2_bedroom_house.csv (161 rows)
[split] Saved: 3_bedroom_house.csv (161 rows)
[split] Saved: 4_bedroom_house.csv (161 rows)
[split] Saved: all_properties.csv (161 rows)


In [24]:
def strip_sheet(in_csv: Path, out_csv: Path) -> int:
    # Read raw with no header so we can promote rows later if needed
    df = pd.read_csv(in_csv, header=None, dtype=str)

    # Need at least 2 rows/cols to drop first row/col safely
    if df.shape[0] >= 2 and df.shape[1] >= 2:
        # Drop first row & first col (typical for these spreadsheets)
        df2 = df.iloc[1:, 1:].reset_index(drop=True)
        # Promote next row to header
        new_header = df2.iloc[0].astype(str).str.strip()
        df2 = df2.iloc[1:].reset_index(drop=True)
        df2.columns = new_header
    else:
        df2 = pd.read_csv(in_csv, dtype=str)

    # Drop unnamed columns, empty rows
    df2 = df2.loc[:, ~df2.columns.str.contains("^Unnamed", case=False)]
    df2 = df2.dropna(how="all")

    # Force first column header to 'Suburb'
    if df2.columns.size > 0:
        df2.columns = ["Suburb"] + df2.columns[1:].tolist()

    # Drop summary rows like "Group Total"
    mask_total = df2.apply(lambda r: r.astype(str).str.contains(r"\bgroup\s*total\b", case=False, regex=True).any(), axis=1)
    df2 = df2.loc[~mask_total]

    df2.to_csv(out_csv, index=False)
    return len(df2)

produced = []
for p in sorted(SPLIT_DIR.glob("*.csv")):
    out = STRIPPED_DIR / p.name
    n = strip_sheet(p, out)
    produced.append(out)
    print(f"[strip] {p.name} → {n} rows")

if not produced:
    print("No stripped files produced — check the split step.")


[strip] 1_bedroom_flat.csv → 147 rows
[strip] 2_bedroom_flat.csv → 147 rows
[strip] 2_bedroom_house.csv → 147 rows
[strip] 3_bedroom_flat.csv → 147 rows
[strip] 3_bedroom_house.csv → 147 rows
[strip] 4_bedroom_house.csv → 147 rows
[strip] all_properties.csv → 147 rows


In [26]:

MONTH_ABBR = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
FILE_RX = re.compile(r"(?P<beds>\d+)_bedroom_(?P<ptype>flat|house)", re.I)

def _clean_cell(x):
    if x is None or (isinstance(x, float) and pd.isna(x)):
        return None
    x = str(x).strip()
    return None if x in {"", "-"} else x

def load_and_build_header(path: Path) -> pd.DataFrame:
    raw = pd.read_csv(path, header=None, dtype=str)
    raw = raw.applymap(lambda x: None if pd.isna(x) else str(x).strip())

    r0 = raw.iloc[0].tolist() if len(raw) > 0 else []
    r1 = raw.iloc[1].tolist() if len(raw) > 1 else []

    def looks_like_two_row_header(r0, r1):
        has_month = any(any(m in str(c) for m in MONTH_ABBR) for c in r0)
        has_metrics = any(str(c).strip().lower() in {"count","median"} for c in r1)
        return has_month and has_metrics

    if looks_like_two_row_header(r0, r1):
        # Forward-fill top row months
        r0_ffill, last = [], None
        for c in r0:
            if c not in [None,"","nan"]:
                last = c
            r0_ffill.append(last)
        cols = []
        for i, (top, sub) in enumerate(zip(r0_ffill, r1)):
            top = "Suburb" if i == 0 else top
            name = top if i == 0 else f"{top}_{sub}"
            cols.append(name)
        df = raw.iloc[2:].copy()
        df.columns = cols
    else:
        df = pd.read_csv(path, dtype=str)
        if not df.columns[0] or str(df.columns[0]).lower().startswith("unnamed"):
            df.rename(columns={df.columns[0]: "Suburb"}, inplace=True)

    df = df.applymap(_clean_cell)
    if "Suburb" not in df.columns:
        df.rename(columns={df.columns[0]: "Suburb"}, inplace=True)
    return df.dropna(how="all")

def wide_to_tidy(df: pd.DataFrame) -> pd.DataFrame:
    val_cols = [c for c in df.columns if c != "Suburb"]
    long = df.melt(id_vars=["Suburb"], value_vars=val_cols,
                   var_name="period_metric", value_name="value")
    ext = long["period_metric"].str.extract(r"(?P<period>[A-Za-z]{3}\s+\d{4})[_\s-]*(?P<metric>Count|Median)?")
    long["period"] = ext["period"]
    long["metric"] = ext["metric"].fillna("")
    tidy = (long.pivot_table(index=["Suburb","period"], columns="metric", values="value", aggfunc="first")
                 .reset_index())
    tidy.columns.name = None
    if "Count" in tidy.columns:
        tidy["Count"] = pd.to_numeric(tidy["Count"], errors="coerce")
    if "Median" in tidy.columns:
        tidy["Median"] = pd.to_numeric(tidy["Median"], errors="coerce")
    tidy["Date"] = pd.to_datetime(tidy["period"], format="%b %Y", errors="coerce")
    tidy = tidy.drop(columns=["period"])
    cols = ["Suburb","Date"] + [c for c in ["Count","Median"] if c in tidy.columns]
    return tidy[cols].sort_values(["Suburb","Date"]).reset_index(drop=True)

def annotate_from_filename(path: Path) -> dict:
    m = FILE_RX.search(path.stem)
    if not m:
        return {"Bedrooms": None, "Dwelling": None}
    return {"Bedrooms": int(m.group("beds")), "Dwelling": ("apartment" if m.group("ptype").lower()=="flat" else m.group("ptype").lower())}

all_frames = []
for p in sorted(STRIPPED_DIR.glob("*.csv")):
    try:
        df = load_and_build_header(p)
        tidy = wide_to_tidy(df)
        meta = annotate_from_filename(p)
        for k,v in meta.items():
            tidy[k] = v
        # reorder
        base = ["Suburb","Date","Bedrooms","Dwelling"]
        vals = [c for c in ["Count","Median"] if c in tidy.columns]
        tidy = tidy[base + vals]
        all_frames.append(tidy)
        # also per-file tidy for inspection
        (TIDY_DIR / f"{p.stem}_tidy.csv").write_text(tidy.to_csv(index=False))
        print(f"[tidy] {p.name:26s} → {len(tidy):5d} rows")
    except Exception as e:
        print(f"[skip] {p.name}: {e}")

if all_frames:
    combo = (pd.concat(all_frames, ignore_index=True)
               .sort_values(["Suburb","Date","Bedrooms","Dwelling"]))
    combo.to_csv(OUT_TIDY, index=False)
    print(f"\n[done] Combined → {OUT_TIDY}  rows={len(combo):,}")
else:
    print("No tidy frames produced — check stripped files.")


/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: None if pd.isna(x) else str(x).strip())
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(_clean_cell)
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: None if pd.isna(x) else str(x).strip())
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(_clean_cell)
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:12: FutureWarning: DataFrame.applymap has been deprecat

[tidy] 1_bedroom_flat.csv         → 13538 rows
[tidy] 2_bedroom_flat.csv         → 14607 rows
[tidy] 2_bedroom_house.csv        → 14124 rows
[tidy] 3_bedroom_flat.csv         → 13978 rows


/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: None if pd.isna(x) else str(x).strip())
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(_clean_cell)
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: None if pd.isna(x) else str(x).strip())
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(_clean_cell)
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:12: FutureWarning: DataFrame.applymap has been deprecat

[tidy] 3_bedroom_house.csv        → 14428 rows
[tidy] 4_bedroom_house.csv        → 14054 rows


/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: None if pd.isna(x) else str(x).strip())
/var/folders/59/n7t8nd2s08zcjsn4bd22nlsr0000gn/T/ipykernel_57050/3719917520.py:41: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(_clean_cell)


[tidy] all_properties.csv         → 14739 rows

[done] Combined → ../../datasets/raw/tidy_monthly/all_properties_tidy.csv  rows=99,468


In [28]:
# 1) Paths
PROJECT_ROOT = Path("../../") 
DATA_ROOT    = PROJECT_ROOT / "datasets"
TIDY_CSV     = DATA_ROOT / "raw/tidy_monthly/all_properties_tidy.csv"
OUT_ENRICHED = DATA_ROOT / "raw/tidy_monthly/all_properties_tidy_enriched.csv"
OUT_UNMATCHED= DATA_ROOT / "raw/tidy_monthly/all_properties_tidy_UNMATCHED_SUBURBS.csv"

MAP_CSV = DATA_ROOT / "district_shape/sa2_lookup/mapped_target_suburbs.csv"

if not MAP_CSV.exists():
    print(f"Mapping not found at:\n  {MAP_CSV}\nTrying to auto-locate it under {PROJECT_ROOT} ...")
    found = list(PROJECT_ROOT.rglob("mapped_target_suburbs.csv"))
    if found:
        MAP_CSV = found[0]
        print(f"Found mapping at:\n  {MAP_CSV}")
    else:
        raise FileNotFoundError(f"Mapping CSV not found. Please check the path above.")

# 2) Helpers
def norm(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.normalize("NFKD").str.encode("ascii","ignore").str.decode("ascii")
         .str.upper().str.strip()
         .str.replace(r"[^\w\s\-]", " ", regex=True)
         .str.replace(r"\s+", " ", regex=True)
    )

# 3) Load data
if not TIDY_CSV.exists():
    raise FileNotFoundError(f"Tidy file not found: {TIDY_CSV}\nRun the tidy build step first.")
df     = pd.read_csv(TIDY_CSV)
map_df = pd.read_csv(MAP_CSV)

# 4) Robust column detection in mapping file
up_map = {c: c.upper() for c in map_df.columns}
def need(name):
    for c,u in up_map.items():
        if u == name:
            return c
    return None

map_suburb_col = need("TARGET_SUBURB") or need("SUBURB") or need("SUBURB_NAME") or need("LOCALITY")
lat_col = need("LAT"); lng_col = need("LNG")
sa2c = need("SA2_CODE21"); sa2n = need("SA2_NAME21")
lgac = need("LGA_CODE21"); lgan = need("LGA_NAME21")

if any(x is None for x in [map_suburb_col, lat_col, lng_col, sa2c, sa2n, lgac, lgan]):
    raise ValueError(f"Mapping CSV missing required columns. Found: {map_df.columns.tolist()}")

# 5) Join on normalized suburb strings
df["_SUBURB_KEY"]     = norm(df["Suburb"])
map_df["_SUBURB_KEY"] = norm(map_df[map_suburb_col])

map_keep = map_df[["_SUBURB_KEY", lat_col, lng_col, sa2c, sa2n, lgac, lgan]].rename(
    columns={
        lat_col:"Lat", lng_col:"Lng",
        sa2c:"SA2_CODE21", sa2n:"SA2_NAME21",
        lgac:"LGA_CODE21", lgan:"LGA_NAME21"
    }
)

enriched = df.merge(map_keep, on="_SUBURB_KEY", how="left").drop(columns=["_SUBURB_KEY"])

# 6) Save + diagnostics
enriched.to_csv(OUT_ENRICHED, index=False)
matched = enriched["SA2_CODE21"].notna().sum()
print(f"Saved: {OUT_ENRICHED} — matched {matched}/{len(enriched)} ({matched/len(enriched):.1%})")

# Unmatched suburb report to fix mapping gaps quickly
unmatched = (
    enriched.loc[enriched["SA2_CODE21"].isna(), "Suburb"]
    .dropna().astype(str).value_counts().reset_index()
)
if not unmatched.empty:
    unmatched.columns = ["Suburb","count"]
    unmatched.to_csv(OUT_UNMATCHED, index=False)
    print(f"Unmatched list saved: {OUT_UNMATCHED}\nTop few:\n", unmatched.head())
else:
    print("No unmatched suburbs ")


Saved: ../../datasets/raw/tidy_monthly/all_properties_tidy_enriched.csv — matched 98774/99468 (99.3%)
Unmatched list saved: ../../datasets/raw/tidy_monthly/all_properties_tidy_UNMATCHED_SUBURBS.csv
Top few:
         Suburb  count
0  Wanagaratta    694
